In [ ]:
import os
import shutil
import organize_files

device = "Spectralis"

current_dir = "C:\\Users\\sanjay\\Developer\\fairhub-pipeline\\spec"

source_data = os.path.join(current_dir, "UW_Spectralis_20231204-20231208.zip") # smaller
# source_data = os.path.join(current_dir, "UW_Spectralis_20241112-20241118.zip") # larger

step_1 = os.path.join(current_dir, "step1")
step_2 = os.path.join(current_dir, "step2")
step_3 = os.path.join(current_dir, "step3")
step_4 = os.path.join(current_dir, "step4")
step_metadata = os.path.join(current_dir, "step_metadata")

In [ ]:
# unzip the source data into the step_1 directory
import zipfile

# delete the step_1 directory if it exists
if os.path.exists(step_1):
    shutil.rmtree(step_1)

with zipfile.ZipFile(source_data, 'r') as zip_ref:
    zip_ref.extractall(step_1)

# go to /DICOM directory in the step_1 directory and add the .dcm extension to all the files.
# currently they have no extension
step_1_dicom_dir = os.path.join(step_1, "DICOM")
for file in os.listdir(step_1_dicom_dir):
    if not file.endswith('.dcm'):
        os.rename(
            os.path.join(step_1_dicom_dir, file),
            os.path.join(step_1_dicom_dir, f'{file}.dcm'),
        )


In [ ]:
# delete the step_2 directory if it exists
if os.path.exists(step_2):
    shutil.rmtree(step_2)

organize_files.process_octa(step_1_dicom_dir, step_2)

In [ ]:
# clean up the dicom file
import pydicom

for root, dirs, files in os.walk(step_2):
    for file in files:
        if file.endswith('.dcm'):
            file_path = os.path.join(root, file)
            
            ds = pydicom.dcmread(file_path)
            
            full_patient_id = ds.PatientID
            patient_id = full_patient_id.split("-")[1]
            
            ds.PatientSex = "M"
            ds.PatientBirthDate = ""
            ds.PatientName = ""
            ds.PatientID = patient_id
            ds.ProtocolName = "spectralis mac 20x20 hs octa"
            
            ds.save_as(file_path)

In [ ]:
import pydicom
from tqdm import tqdm

# Generate a dict with all the data needed to organize the step_2 data into step_3
file_list = []

for root, dirs, files in os.walk(step_2):
    for file in files:
        if file.endswith('.dcm'):
            file_path = os.path.join(root, file)
            file_name = os.path.basename(file_path)
            file_name = file_name.split(".")[0]
            
            image_type = file_name.split("_")[1]
            
            ds = pydicom.dcmread(file_path)
            
            patient_id = ds.PatientID
            laterality = ds.Laterality.lower()
            sop_instance_uid = ds.SOPInstanceUID
            
            file_list.append({
                "file_path": file_path,
                "patient_id": patient_id,
                "laterality": laterality,
                "sop_instance_uid": sop_instance_uid,
                "image_type": image_type
            })

# delete the step_3 directory if it exists
if os.path.exists(step_3):
    shutil.rmtree(step_3)

# organize the step_2 data into step_3
image_type_mapping = {
    "enface":  { 
        "label" : "enface",
        "path" : ["retinal_octa", "enface", "heidelberg_spectralis"]},
    "heightmap": { 
        "label" : "segmentation",
        "path" : ["retinal_octa", "segmentation", "heidelberg_spectralis"]},
    "vol": { 
        "label" : "flow_cube",
        "path" : ["retinal_octa", "flow_cube", "heidelberg_spectralis"]},
    "op": { 
        "label" : "ir",
        "path" : ["retinal_photography", "ir", "heidelberg_spectralis"]},
    "opt": { 
        "label" : "oct",
        "path" : ["retinal_oct", "structural_oct", "heidelberg_spectralis"]},
}

# copy the files to the step_3 directory using tqdm
for file in tqdm(file_list):
    file_path = file["file_path"]
    patient_id = file["patient_id"]
    laterality = file["laterality"]
    sop_instance_uid = file["sop_instance_uid"]
    image_type = file["image_type"]

    mapped_image_type = image_type_mapping[image_type]

    output_dir = os.path.join(step_3, os.path.join(*mapped_image_type["path"]), patient_id)
    
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)
    
    new_file_name = f"{patient_id}_spectralis_mac_20x20_hs_octa_{mapped_image_type["label"]}_{laterality}_{sop_instance_uid}.dcm"
    
    new_file_path = os.path.join(output_dir, new_file_name)
    
    shutil.copy(file_path, new_file_path)

enface - retinal_octa/enface/heidelberg_spectralis
op - retinal_photography/ir/heidelberg_spectralis
opt - retinal_oct/oct_structural_scan/heidelberg_spectralis
vol - retinal_octa/flow_cube/heidelberg_spectralis
heightmap - retinal_octa/segmentation/heidelberg_spectralis

device = spectralis
protocol = spectralis_mac_20x20_hs_octa

patientid_device+protocol+laterality+sopinstanceuid.dcm
patientid_device+protocol_laterality_sopinstanceuid.dcm

initial_file_path = "C:\\Users\\sanjay\\Developer\\fairhub-pipeline\\spec\\step3\\AIREADI-xxxx_R_spectralis_mac_20x20_hs_octa_oct_x.xx.xxx\\xxx_heightmap.dcm"

final_file_path = "xxxx_spectralis_mac_20x20_hs_octa_r_x.xx.xxx.dcm"